In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython
model = MatrixFactorization_BPR_Cython

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2537 (20.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [5]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam"])
    epochs = 1500
    num_factors = trial.suggest_int("num_factors", 10, 300)
    batch_size = trial.suggest_categorical("batch_size", [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024])
    positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
    negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [7]:
best_params = {
    "sgd_mode": "adam",
    "epochs": 240,
    "num_factors": 148,
    "batch_size": 16,
    "positive_reg": 0.000439293634625917,
    "negative_reg": 5.495056923670675e-05,
    "learning_rate": 0.0007794513725937752
}

# {"sgd_mode": "adam", "epochs": 240, "num_factors": 148, "batch_size": 16, "positive_reg": 0.000439293634625917, "negative_reg": 5.495056923670675e-05, "learning_rate": 0.0007794513725937752}
study_name = "MFACT_BPR-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

[I 2023-11-22 15:34:42,109] A new study created in memory with name: no-name-b08f23a4-07aa-4c53-889b-856c0c29a715
/tmp/ipykernel_7190/1271262102.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  positive_reg = trial.suggest_loguniform("positive_reg", 1e-6, 1e-1)
/tmp/ipykernel_7190/1271262102.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  negative_reg = trial.suggest_loguniform("negative_reg", 1e-6, 1e-1)
/tmp/ipykernel_7190/1271262102.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = t

MatrixFactorization_BPR_Cython_Recommender: URM Detected 478 ( 3.8%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 318 ( 1.4%) items with no interactions.


[W 2023-11-22 15:34:42,507] Trial 0 failed with parameters: {'sgd_mode': 'adam', 'num_factors': 148, 'batch_size': 16, 'positive_reg': 0.000439293634625917, 'negative_reg': 5.495056923670675e-05, 'learning_rate': 0.0007794513725937752} because of the following error: AttributeError("module 'numpy' has no attribute 'int'.\n`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.\nThe aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:\n    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations").
Traceback (most recent call last):
  File "/home/alessandro/RecSys-2023-polimi/venv/lib/python3.8/site-packages/optuna/st

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [9]:
study.best_params

{'topK': 101, 'alpha': 0.3026342852596128, 'beta': 0.058468783118329024}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params)

RP3betaRecommender: URM Detected 245 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 110 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4027.41 column/sec. Elapsed time 5.52 sec


In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2100 (16.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10538 (100.0%) in 5.18 sec. Users per second: 2033


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.090378                 0.157221  0.133194  0.045665    0.078973   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.259414  0.143182  0.107686  0.509869      0.339649  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.833834          0.425146    0.833834       0.057877   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.145296                   0.997617             0.165954   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   